In [1]:
import cv2 
import dlib
from imutils import face_utils
from scipy.spatial import distance

In [2]:
count = 0
close = 0
morse_word = ''
flag1 = False
flag2 = False
flag3 = False
sentence = ''

In [3]:
MORSE_CODE_DICT = { 'A':'.-',         'B':'-...',
                    'C':'-.-.',       'D':'-..',          'E':'.',
                    'F':'..-.',       'G':'--.',          'H':'....',
                    'I':'..',         'J':'.---',         'K':'-.-',
                    'L':'.-..',       'M':'--',           'N':'-.',
                    'O':'---',        'P':'.--.',         'Q':'--.-',
                    'R':'.-.',        'S':'...',          'T':'-',
                    'U':'..-',        'V':'...-',         'W':'.--',
                    'X':'-..-',       'Y':'-.--',         'Z':'--..',
                    '1':'.----',      '2':'..---',        '3':'...--',
                    '4':'....-',      '5':'.....',        '6':'-....',
                    '7':'--...',      '8':'---..',        '9':'----.',
                    '0':'-----',      ', ':'--..--',      '.':'.-.-.-',
                    '?':'..--..',     '/':'-..-.',        '-':'-....-',
                    '(':'-.--.',      ')':'-.--.-'}

In [4]:
key_list = list(MORSE_CODE_DICT.keys())
val_list = list(MORSE_CODE_DICT.values())

In [5]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [6]:
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])

    C = distance.euclidean(eye[0], eye[3])
    eye = (A + B) / (2.0 * C)

    return eye

In [7]:
cap = cv2.VideoCapture(0)

while True:
    _, image = cap.read()
    imgGray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) 
    faces = detector(imgGray)
    
    for face in faces:
        landmarks = predictor(imgGray,face)

        landmarks = face_utils.shape_to_np(landmarks)

        
        leftEye = landmarks[42:48]
        rightEye = landmarks[36:42]

        for i in leftEye:
            cv2.circle(image, (i), 2, (255,255,255), thickness=1)
        for i in rightEye:
            cv2.circle(image, (i), 2, (255,255,255), thickness=1)
            
        leftEye = eye_aspect_ratio(leftEye)
        rightEye = eye_aspect_ratio(rightEye)

        eye = ((leftEye + rightEye) / 2.0)
        
        if eye <= 0.3:
            count += 1
            #print('count: ',count)
            if count >= 4:
                close += 1
                cv2.putText(image, "Eye Close", (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                            
        else:
            if close >= 15 and close < 60:
                flag1 = True
                count = 0
                
            elif close >= 60 and close <= 120:
                flag2 = True
                count = 0
                
            elif close >= 120:
                flag3 = True
                count = 0
                
                

    if flag1: 
        morse_word += '.'
        flag1 = False
        close = 0
        print(morse_word)
        
    elif flag2: 
        morse_word += '-'
        flag2 = False
        close = 0
        print(morse_word)
        
    elif flag3 and morse_word not in val_list:
        flag3 = False
        close = 0
        print('No letter for the morse code',morse_word)
        morse_word = ''
        
    elif flag3: 
        position = val_list.index(morse_word)
        sentence += key_list[position]
        flag3 = False
        close = 0
        morse_word = ''
        print('The word is',sentence)
    
    cv2.imshow('window', image)
    
    k = cv2.waitKey(1)
    
    if k == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

-
--
The word is M
.
.-
The word is MA
-
-.
The word is MAN
.
..
The word is MANI
.
..
...
....
.....
......
.......
No letter for the morse code 
